## Analiza cijena goriva u Hrvatskoj, Sloveniji i Mađarskoj za 2022. godinu

### Učitavanje i čitanje podataka

In [36]:
import pandas as pd

In [37]:
df = pd.read_excel('E:/cijene/Cijene.xlsx')

In [38]:
df.head()

,Prices in force on,Country Name,Country EU Code,Product Name,Currency Code,Prices Unit,Euro exchange rate,Weekly price with taxes,Weekly price without taxes
0,2022-01-03,Croatia,HR,Automotive gas oil,HRK,1000L,0.132996,"1,463.49",763.82
1,2022-01-03,Croatia,HR,Euro-super 95,HRK,1000L,0.132996,"1,484.9",674.56
2,2022-01-03,Croatia,HR,Fuel oil - Sulphur less than 1%,HRK,t,0.132996,712.73,691.45
3,2022-01-03,Croatia,HR,Heating gas oil,HRK,1000L,0.132996,756.75,559.78
4,2022-01-03,Croatia,HR,LPG - motor fuel,HRK,1000L,0.132996,786.27,621.65


## Informacije o podacima

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Prices in force on          714 non-null    datetime64[ns]
 1   Country Name                714 non-null    object        
 2   Country EU Code             714 non-null    object        
 3   Product Name                714 non-null    object        
 4   Currency Code               714 non-null    object        
 5   Prices Unit                 714 non-null    object        
 6   Euro exchange rate          714 non-null    float64       
 7   Weekly price with taxes     714 non-null    object        
 8   Weekly price without taxes  714 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 50.3+ KB


### Transformacija i čišćenje podataka

In [40]:
#Najprije je potrebno prebaciti stupac sa cijenama u float te ukloniti zareze u odvajanu tisućica
df['Weekly price with taxes'] = df['Weekly price with taxes'].str.replace(',', '').astype(float)
df['Weekly price without taxes'] = df['Weekly price without taxes'].str.replace(',', '').astype(float)

In [41]:
#Zamjena mjerne jedinica tako da svuda piše 1000L
df['Prices Unit'] = df['Prices Unit'].replace({'t': '1000L'})
df.to_excel("E:/Podaci za tableau/Cijenetransofrmirano.xlsx", index=False)

In [42]:
df.head()

,Prices in force on,Country Name,Country EU Code,Product Name,Currency Code,Prices Unit,Euro exchange rate,Weekly price with taxes,Weekly price without taxes
0,2022-01-03,Croatia,HR,Automotive gas oil,HRK,1000L,0.132996,1463.49,763.82
1,2022-01-03,Croatia,HR,Euro-super 95,HRK,1000L,0.132996,1484.90,674.56
2,2022-01-03,Croatia,HR,Fuel oil - Sulphur less than 1%,HRK,1000L,0.132996,712.73,691.45
3,2022-01-03,Croatia,HR,Heating gas oil,HRK,1000L,0.132996,756.75,559.78
4,2022-01-03,Croatia,HR,LPG - motor fuel,HRK,1000L,0.132996,786.27,621.65


## Izračun prosječne, maksimalne i minimalne cijene

In [43]:

# Grupiranje dataframe-a prema državi, vrsti goriva, te izračun prosječne, minimalne i maksimalne cijene goriva na određeni datum
# Funkcija za pridruživanje datuma max i min cijenama
def get_max_min_dates(group):
    max_price_idx = group['Weekly price with taxes'].idxmax()
    min_price_idx = group['Weekly price with taxes'].idxmin()
    group['Max na datum'] = group.loc[max_price_idx, 'Prices in force on']
    group['Min na datum'] = group.loc[min_price_idx, 'Prices in force on']
    return group

# Grupiranje i primjena funkcije
result = df.groupby(['Country Name', 'Product Name'],group_keys=False).apply(get_max_min_dates)

# Izračun prosječne, max i min cijene
result = result.groupby(['Country Name', 'Product Name']).agg({
    'Weekly price with taxes': ['mean', 'max', 'min'],
    'Max na datum': 'first',  # Prva vrijednost nakon grupiranja je ona koja je pridružena max cijeni
    'Min na datum': 'first'   # Prva vrijednost nakon grupiranja je ona koja je pridružena min cijeni
}).reset_index()

result.columns = ['Country Name', 'Product Name', 'Prosječna cijena', 'Max Cijena', 'Min Cijena', 'Max na datum', 'Min na datum']

result.to_excel("E:/Podaci za tableau/Prosj_max_min_sve_drz.xlsx", index=False)

result

,Country Name,Product Name,Prosječna cijena,Max Cijena,Min Cijena,Max na datum,Min na datum
0,Croatia,Automotive gas oil,1762.536078,1947.25,1463.49,2022-07-04,2022-01-03
1,Croatia,Euro-super 95,1604.532549,1873.26,1331.17,2022-06-06,2022-12-26
2,Croatia,Fuel oil - Sulphur less than 1%,922.257647,1110.35,711.89,2022-03-21,2022-01-10
3,Croatia,Heating gas oil,1188.309200,1471.53,756.75,2022-07-11,2022-01-03
4,Croatia,LPG - motor fuel,944.967255,1050.85,786.27,2022-06-13,2022-01-03
5,Hungary,Automotive gas oil,1478.166078,1778.38,1220.32,2022-12-26,2022-03-07
6,Hungary,Euro-super 95,1289.545882,1585.74,1202.99,2022-12-19,2022-10-10
7,Hungary,Fuel oil - Sulphur less than 1%,815.240980,960.96,559.97,2022-06-06,2022-01-24
8,Hungary,Heating gas oil,1478.166078,1778.38,1220.32,2022-12-26,2022-03-07
9,Hungary,LPG - motor fuel,954.940000,1022.27,839.89,2022-08-08,2022-03-07


In [44]:
# Izračun minimalne i maksimalne cijene prema vrsti goriva
max_prices = df.groupby('Product Name')['Weekly price with taxes'].idxmax()
min_prices = df.groupby('Product Name')['Weekly price with taxes'].idxmin()

# Izračun datuma
max_price_df = df.loc[max_prices, ['Product Name', 'Country Name', 'Weekly price with taxes', 'Prices in force on']]
min_price_df = df.loc[min_prices, ['Product Name', 'Country Name', 'Weekly price with taxes', 'Prices in force on']]


max_price_df.columns = ['Product Name', 'Država sa Max Cijenom', 'Max Cijena', 'Datum Max Cijene']
min_price_df.columns = ['Product Name', 'Država sa Min Cijenom', 'Min Cijena', 'Datum Min Cijene']

#Resetiranje indexa
max_price_df.reset_index(drop=True, inplace=True)
min_price_df.reset_index(drop=True, inplace=True)


result = pd.merge(max_price_df, min_price_df, on='Product Name')

result.to_excel("E:/Podaci za tableau/Prosj_max_min_za_goriva.xlsx", index=False)
result

,Product Name,Država sa Max Cijenom,Max Cijena,Datum Max Cijene,Država sa Min Cijenom,Min Cijena,Datum Min Cijene
0,Automotive gas oil,Croatia,1947.25,2022-07-04,Hungary,1220.32,2022-03-07
1,Euro-super 95,Croatia,1873.26,2022-06-06,Hungary,1202.99,2022-10-10
2,Fuel oil - Sulphur less than 1%,Croatia,1110.35,2022-03-21,Hungary,559.97,2022-01-24
3,Heating gas oil,Hungary,1778.38,2022-12-26,Croatia,756.75,2022-01-03
4,LPG - motor fuel,Slovenia,1054.42,2022-04-25,Croatia,786.27,2022-01-03


## Promjena cijena kroz godinu za svaku državu prema vrsti

In [49]:

# Sortiranje podataka prema državi, vrsti goriva i datumu
df_promjena = df.sort_values(['Country Name', 'Product Name', 'Prices in force on'])

# Izračunavanje promjena cijena kroz godinu za svaku državu i vrstu goriva
df_promjena['Promjena cijene'] = df_promjena.groupby(['Country Name', 'Product Name'])['Weekly price with taxes'].diff()



In [50]:
df_promjena.head()

,Prices in force on,Country Name,Country EU Code,Product Name,Currency Code,Prices Unit,Euro exchange rate,Weekly price with taxes,Weekly price without taxes,Promjena cijene
0,2022-01-03,Croatia,HR,Automotive gas oil,HRK,1000L,0.132996,1463.49,763.82,NaN
5,2022-01-10,Croatia,HR,Automotive gas oil,HRK,1000L,0.132841,1465.50,765.91,2.01
10,2022-01-17,Croatia,HR,Automotive gas oil,HRK,1000L,0.132846,1492.53,787.51,27.03
15,2022-01-24,Croatia,HR,Automotive gas oil,HRK,1000L,0.132820,1519.99,809.56,27.46
20,2022-01-31,Croatia,HR,Automotive gas oil,HRK,1000L,0.132814,1520.19,809.74,0.20


## Izračun korelacijske matrice

In [59]:
grupiranje_po_gorivu = df.groupby(['Product Name', 'Prices in force on'])['Weekly price with taxes'].mean().reset_index()

# Pivotiranje podataka tako da svaka vrsta goriva postane zaseban stupac
pivot_df = grupiranje_po_gorivu.pivot(index='Prices in force on', columns='Product Name', values='Weekly price with taxes')

# Izračun korelacijske matrice
correlation_matrix = pivot_df.corr()
correlation_matrix.to_excel("E:/Podaci za tableau/korelacijska_matrica.xlsx")
correlation_matrix

Product Name,Automotive gas oil,Euro-super 95,Fuel oil - Sulphur less than 1%,Heating gas oil,LPG - motor fuel
Product Name,,,,,
Automotive gas oil,1.000000,0.351173,0.786315,0.965548,0.821554
Euro-super 95,0.351173,1.000000,0.483631,0.441623,0.577598
Fuel oil - Sulphur less than 1%,0.786315,0.483631,1.000000,0.803224,0.844689
Heating gas oil,0.965548,0.441623,0.803224,1.000000,0.873595
LPG - motor fuel,0.821554,0.577598,0.844689,0.873595,1.000000


### Korištenje Python skripte u Tableau pomoću TabPy-a

In [1]:
from tabpy_tools.client import Client

def izracun_medijana(_arg1):
    import numpy as np
    medijan = np.median(_arg1)
    return medijan.tolist()

client = Client('http://localhost:9004/')
client.deploy('IzracunajMedijan', izracun_medijana, 'Racuna medijalnu vrijednost', override=True)

ModuleNotFoundError: No module named 'tabpy_tools'